In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import Adam, RMSprop
from torch.utils.data import DataLoader
import torchvision.transforms as T

import os
from time import time
import random
import shutil
from datetime import datetime
from collections import defaultdict

from models.unet import MobilenetV2, Resnet34_upsample, Resnet50, InceptionResnetV2
from loss import BCEDiceLoss, BCEJaccardLoss

pjoin = join = os.path.join

In [3]:
random.seed(2018)
np.random.seed(2018)
torch.manual_seed(2018)
torch.cuda.manual_seed_all(2018)

In [4]:
cv2.setNumThreads(0)

In [5]:
TRAIN_ROOT = './data/AerialImageDataset_crop/train/'
IMAGES_ROOT = TRAIN_ROOT + 'images'
GT_ROOT = TRAIN_ROOT + 'gt'

OUTPUT_DIR = './output/'

IMAGE_SZ = 512
ORIG_IMAGE_SZ = 1250

In [6]:
out_ch = 1

criterion = BCEDiceLoss()

model_class = Resnet34_upsample

def get_model():
    model = nn.DataParallel(model_class(out_ch, 3))
    model.cuda()
    return model

if model_class in (MobilenetV2, Resnet34_upsample, Resnet50):
    pretrained_settings = {'mean': [0.485, 0.456, 0.406], 'std': [0.229, 0.224, 0.225]}
elif model_class in (InceptionResnetV2,):
    pretrained_settings = {'mean': [0.5, 0.5, 0.5], 'std': [0.5, 0.5, 0.5]}

In [7]:
pretrained_settings

{'mean': [0.485, 0.456, 0.406], 'std': [0.229, 0.224, 0.225]}

In [8]:
class Progress:
    def __init__(self, total, desc, print_freq=0.1):
        self.total = total
        self.desc = desc
        self.prog = 0
        self.last_print = 0
        self.postfix = ''
        self.print_freq = print_freq
                
    def update(self, step):
        self.prog += step
        if self.prog == self.total:
            self._print()
        if self.last_print / self.total < self.print_freq \
            and (self.last_print + step) / self.total >= self.print_freq:
            self._print()
            self.last_print = 0
        else:
            self.last_print += step
            
    def _print(self):
        prog = self.prog / self.total * 100
        line = f'{self.desc} {prog:.2f}% {self.postfix}'
        print(line)
        
    def set_postfix(self, postfix):
        self.postfix = str(postfix)
        

class Logger:
    def __init__(self, log_filepath):
        self.log = open(log_filepath, 'w')

    def msg(self, msg, output_to_console=False, tq=None):
        self.log.write('%s %s\n' % (datetime.now().isoformat(), msg))
        if output_to_console:
            if tq is not None:
                tq.write(msg)
            else:
                print(msg)
                
    def __del__(self):
        self.log.close()
        
        
def ocv_loader(fpath):
    im = cv2.imread(fpath)
    return im[:, :, ::-1].copy() if im is not None else None


def make_dir(path, remove_exists=False):
    if os.path.exists(path):
        print(f'WARNING: {path} exists')
        if remove_exists:
            shutil.rmtree(path)
    try:
        os.mkdir(prediction_output_dir)
    except FileExistsError:
        pass
        

normalizer = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=pretrained_settings['mean'], std=pretrained_settings['std']),
])


class Denormalize:
    def __init__(self, mean, std):
        self.mean = torch.FloatTensor(mean)
        self.std = torch.FloatTensor(std)
        
    def __call__(self, img):
        img = img.permute(1, 2, 0)
        return img * self.std + self.mean


denormalizer = Denormalize(pretrained_settings['mean'], pretrained_settings['std'])


def mask_overlay(image, mask, ch=0):
    if not isinstance(mask, np.ndarray):
        mask = mask.numpy()
    if mask.ndim > 2:
        mask = mask[ch,:,:]
    if isinstance(image, str):
        im = cv2.imread(image)
        im = cv2.resize(im, (mask.shape[1], mask.shape[0]))
    else:
        im = denormalizer(image).numpy()
        im = (im * 255).astype(np.uint8)
    mask_ch = np.clip((255*mask) + im[...,0], 0, 255).astype(np.uint8)
    return np.dstack((mask_ch,im[...,1],im[...,2]))


def variable(x, volatile=False):
    return Variable(x, volatile=volatile).cuda()
    
    
def pad_image(img, padw, padh):
    pads = [[0, padw], [0, padh]]
    if img.ndim == 3:
        pads += [[0, 0]]
    return np.pad(img, pads, 'symmetric')


def fix_input_size(img):
    assert img.shape[0] == img.shape[1]
    if model_class == UNetInceptionResnetV2:
        pad = 31 - img.shape[0] % 32
    else:
        pad = 32 - img.shape[0] % 32
    return pad_image(img, pad, pad)


def prepare_data(img, mask):
    img = normalizer(img)
    mask = torch.from_numpy(mask / 255).unsqueeze(0).float()
    return img, mask


def prepare_img(img, eval_mode=True):
    img = normalizer(img)
    if len(img.shape) == 3:
        img = img.unsqueeze(0)
    return variable(img, volatile=eval_mode)


def output2pred(output, ch=0):
    return torch.sigmoid(output).data[0].cpu().numpy()[ch]
#     return torch.exp(pred).data[0].cpu().numpy()[ch]

    
##### VISUALIZE #####

def imshow(im,figsz=(12,12),**kwargs):
    plt.figure(figsize=figsz)
    plt.imshow(im,**kwargs)
    plt.axis('off')
    
    
def predshow(pred, ch=0):
    imshow(pred2img(pred, ch),cmap='gray',vmin=0, vmax=1)

In [9]:
class RandomCrop(object):
    def __init__(self, size):
        if isinstance(size, int):
            self.size = (int(size), int(size))
        else:
            self.size = size

    def __call__(self, img, mask=None):
        h, w = img.shape[:2]
        tw, th = self.size
        if w == tw and h == th:
            return img, mask

        if ((w - tw) > 0) and ((h - th) > 0):
            x1 = np.random.randint(0, w - tw)
            y1 = np.random.randint(0, h - th)
        else:
            x1 = 0
            y1 = 0
        img = img[y1:y1 + th, x1:x1 + tw]
        if mask is not None:
            mask = mask[y1:y1 + th, x1:x1 + tw]
        return img, mask
    

class CenterCrop:
    def __init__(self, size):
        if isinstance(size, (list, tuple, np.ndarray)):
            self.width, self.height = size
        else:
            self.width = self.height = size

    def __call__(self, img, mask=None):
        h, w, c = img.shape
        dx = (w-self.width)//2
        dy = (h-self.height)//2

        y1 = dy
        y2 = y1 + self.height
        x1 = dx
        x2 = x1 + self.width
        img = img[y1:y2, x1:x2]
        if mask is not None:
            mask = mask[y1:y2, x1:x2]
        return img, mask
    
    
# ### IMGAUG ###
import imgaug as ia
import imgaug.augmenters as iaa
    
    
sometimes = lambda aug: iaa.Sometimes(0.5, aug)

# GEOMETRY
ia_aug_geom_light = iaa.Sequential([
    iaa.Fliplr(0.5),
    sometimes(iaa.Affine(
        rotate=(-45, 45),
        order=1
    ))
], random_order=True)

ia_aug_geom = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.Flipud(0.5),
    sometimes(iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        rotate=(-45, 45),
        shear=(-5, 5),
        order=1
    )),
    sometimes(iaa.OneOf([
        iaa.PiecewiseAffine(scale=(0.01, 0.02)),
        iaa.PerspectiveTransform(scale=(0.01, 0.1))
    ])),
], random_order=True)

# COLOR
ia_aug_color_light = iaa.Sequential([
    iaa.Add((-10, 10)),
    iaa.ContrastNormalization((0.8, 1.2))
], random_order=True)
 
ia_aug_color = iaa.Sequential([
    iaa.Add((-10, 10)),
    sometimes(iaa.ContrastNormalization((0.8, 1.2))),
    sometimes(iaa.AddToHueAndSaturation((-5, 5))),
    sometimes(iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.01*255))),
    sometimes(iaa.Sharpen(alpha=(0, 0.5), lightness=(0.75, 1.5))),
], random_order=True)


def augment(img, mask):
    aug_geom = ia_aug_geom.to_deterministic()
    aug_color = ia_aug_color
    img = aug_geom.augment_image(img)
    img = aug_color.augment_image(img)
    mask = aug_geom.augment_image(mask)
    return img.copy(), mask.copy()

def dummy_augment(img, mask):
    return img, mask

In [10]:
def train_val_split(cities=None):
    def get_city_n(s):
        for i, c in enumerate(s):
            if c.isdigit():
                break
        return int(s[i:])

    train_fnames, val_fnames = [], []
    for fname in os.listdir(IMAGES_ROOT):
        toks = fname[:-4].split('-')
        if cities:
            if not sum([x in toks[0] for x in cities]):
                continue
        city_n = '-'.join(toks[:-1])
        part = toks[-1]
        if get_city_n(city_n) < 6:
            val_fnames.append(fname)
        else:
            train_fnames.append(fname)
    return train_fnames, val_fnames


class Dataset:
    def __init__(self, image_sz, image_list, augment=dummy_augment, epoch_mul=1):
        self.image_sz = image_sz
        self.image_list = image_list
        self.epoch_mul = epoch_mul
        self.augment = augment
    
    def __getitem__(self, idx):
        idx %= len(self.image_list)            
        image_path = pjoin(IMAGES_ROOT, self.image_list[idx])
        gt_path = pjoin(GT_ROOT, self.image_list[idx])
        img = ocv_loader(image_path)
        gt = ocv_loader(gt_path)[:, :, 0]
        assert img is not None
        assert gt is not None
        
        pre_aug_sz = int(self.image_sz * 1.5)
        img, gt = RandomCrop(pre_aug_sz)(img, gt)
        img, gt = self.augment(img, gt)
        img, gt = CenterCrop(self.image_sz)(img, gt)
        img, gt = prepare_data(img, gt)
        
        return img, gt, image_path
    
    def __len__(self):
        return len(self.image_list) * self.epoch_mul

In [11]:
# def calc_loss(model, criterion, file_names, return_list=False):
#     model.eval()
#     losses = []
#     for i, fname in enumerate(file_names):
#         image_path = pjoin(IMAGES_ROOT, fname)
#         gt_path = pjoin(GT_ROOT, fname)
#         img = ocv_loader(image_path)
#         gt = ocv_loader(gt_path)[:, :, 0]
#         img, gt = fix_shape(img), fix_shape(gt)
#         img, gt = one_batch(img, gt)
#         output = model(img)
#         losses.append(criterion(output, gt).data[0])
#     return np.mean(losses)


class CyclicLR:
    def __init__(self, start_lr, cycle_iters=20000, increase_1cycle=1.):
        self.start_lr = start_lr
        self.cycle_iters = cycle_iters
        self.increase_1cycle = increase_1cycle
        self.n_cycle = -1
        
    def calc_lr(self, it, callback=None):
        cycle_it = it % self.cycle_iters
        if cycle_it == 0:
            self.n_cycle += 1
            if it and callback:
                callback(str(it))
        x = .5 * (1 + np.cos(np.pi * cycle_it / (self.cycle_iters - 1)))
        if self.n_cycle == 0:
            x *= self.increase_1cycle
        return x * self.start_lr


class StepLR:
    def __init__(self, start_lr, lr_decay_iters=20000, lr_decay=1./np.sqrt(10)):
        self.start_lr = start_lr
        self.lr_decay = lr_decay
        self.lr_decay_iters = lr_decay_iters
        self.lr_decay_n = 0
        
    def calc_lr(self, it, callback=None):
        if it and it % self.lr_decay_iters == 0:
            self.lr_decay_n += 1
            if callback:
                callback(str(it))
        return self.start_lr * self.lr_decay ** self.lr_decay_n

In [12]:
def train(logger,
          model,
          output_dirpath,
          optimizer,
          num_iters,
          criterion,
          train_dataloader, 
          val_set,
          lr_policy,
          start_iter=None):
    def save_snapshot(suffix):
        model_dirpath = join(output_dirpath, f'model_{suffix}.pt')
        torch.save(model.state_dict(), model_dirpath)
        
    def set_lr(lr):
        for i, grp in enumerate(optimizer.param_groups):
            grp['lr'] = lr
        
    save_each_iter = 20000
    
    logger.msg(str(model))
    train_hist = defaultdict(list)
    best_val_loss = float('inf')
    it = 0 if start_iter is None else start_iter
    try:
        while it < num_iters:
            model.train()
            for inputs, targets, _ in train_dataloader:
                inputs, targets = variable(inputs), variable(targets)
                outputs = model(inputs)
                
#                 bce_batch_weigth = targets.clone() * fg_weight
#                 loss = criterion(outputs, targets, bce_batch_weight=bce_batch_weigth)
                loss = criterion(outputs, targets)
                
                lr = lr_policy.calc_lr(it, save_snapshot)
                set_lr(lr)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                train_loss = loss.data[0]
                if it % 20 == 0:
                    logger.msg(f'Stage: train, iter: {it}, lr: {lr:.3e}, loss {train_loss:.6f}',
                              output_to_console=True)
                train_hist['train'].append((it, train_loss))
                
#                 if it % save_each_iter == 0:
#                     save_snapshot(str(it))
                
#                 if it % 5000 == 0:
#                 #if it and it % 5 == 0:
#                     save_snapshot(it)
                    
#                     if it % 10000 == 0:
#                     #if it and it % 10 == 0:
#                         val_loss = calc_loss(model, criterion, val_set)
#                         logger.msg(f'Stage: eval, iter: {it}, loss {val_loss:.6f}',
#                                   output_to_console=True)
#                         train_hist['val'].append((it, val_loss))
#                         if val_loss < best_val_loss:
#                             best_val_loss = val_loss
#                             best_model_dirpath = join(output_dirpath, 'model_best.pt')
#                             shutil.copy(model_dirpath, best_model_dirpath)

                        
                it += 1
                if it == num_iters:
                    break
    except KeyboardInterrupt:
        torch.save(model.state_dict(), model_dirpath)
        print('kb interrupt')
    return train_hist

In [13]:
!nvidia-smi

Mon Jun 25 00:02:47 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.26                 Driver Version: 396.26                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:17:00.0 Off |                  N/A |
| 22%   57C    P0    60W / 250W |      0MiB / 11178MiB |      1%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:65:00.0 Off |                  N/A |
| 29%   55C    P8    19W / 250W |   6129MiB / 11175MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [14]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [15]:
batch_size = 8
epoch_mul = 1

# >>> CHECK INCLUDED CITIES <<<
# cities = ['vienna']
cities = []
train_set, val_set = train_val_split(cities)

ds_train = Dataset(IMAGE_SZ, train_set, augment=augment, epoch_mul=epoch_mul)
ds_valid = Dataset(IMAGE_SZ, val_set, epoch_mul=epoch_mul)

dl_train = DataLoader(ds_train, batch_size=batch_size, shuffle=True, num_workers=8)
dl_valid = DataLoader(ds_valid, batch_size=batch_size, shuffle=False, num_workers=4)

In [16]:
len(train_set), len(val_set)

(2480, 400)

In [17]:
experiment_name = 'res50_i512_ch128_freeze_encoder'
experiment_dir = OUTPUT_DIR + experiment_name
#!rm -r {experiment_dir}
!mkdir -p {experiment_dir}

In [18]:
logger = Logger(pjoin(experiment_dir, 'train_log.txt'))

model = get_model()

start_lr = 1e-4
optimizer_freeze_encoder = Adam([
    {'params': model.module.encoder_squeeze.parameters()},
    {'params': model.module.decoder_stages.parameters()},
    {'params': model.module.bottlenecks.parameters()},
    {'params': model.module.last_upsample.parameters()},
    {'params': model.module.final.parameters()}
], lr=start_lr)
lr_policy = StepLR(start_lr)

# model.load_state_dict(torch.load('./output/irv2_i384_b16_upsample_dice2/model_last.pt'))
train_dict = {
    'model': model,
    'output_dirpath': experiment_dir,
    'optimizer': optimizer_freeze_encoder,
    'lr_policy': lr_policy,
#     'start_iter': 200000, 
    'num_iters': 10000,
    'criterion': criterion,
    'train_dataloader': dl_train,
    'val_set': val_set,
    'logger': logger,
}
train_hist = train(**train_dict)

model_dirpath = join(experiment_dir, f'model_last.pt')
torch.save(model.state_dict(), model_dirpath)

Stage: train, iter: 0, lr: 1.000e-04, loss 10.072884
Stage: train, iter: 20, lr: 1.000e-04, loss 0.870269
Stage: train, iter: 40, lr: 1.000e-04, loss 0.509665
Stage: train, iter: 60, lr: 1.000e-04, loss 0.555592
Stage: train, iter: 80, lr: 1.000e-04, loss 0.579035
Stage: train, iter: 100, lr: 1.000e-04, loss 0.398833
Stage: train, iter: 120, lr: 1.000e-04, loss 0.451560
Stage: train, iter: 140, lr: 1.000e-04, loss 0.462909
Stage: train, iter: 160, lr: 1.000e-04, loss 0.375812
Stage: train, iter: 180, lr: 1.000e-04, loss 0.447462
Stage: train, iter: 200, lr: 1.000e-04, loss 0.378507
Stage: train, iter: 220, lr: 1.000e-04, loss 0.357986
Stage: train, iter: 240, lr: 1.000e-04, loss 0.633157
Stage: train, iter: 260, lr: 1.000e-04, loss 0.409857
Stage: train, iter: 280, lr: 1.000e-04, loss 0.438029
Stage: train, iter: 300, lr: 1.000e-04, loss 0.417497
Stage: train, iter: 320, lr: 1.000e-04, loss 0.306099
Stage: train, iter: 340, lr: 1.000e-04, loss 0.268405
Stage: train, iter: 360, lr: 1.00

Stage: train, iter: 3000, lr: 1.000e-04, loss 0.275719
Stage: train, iter: 3020, lr: 1.000e-04, loss 0.326566
Stage: train, iter: 3040, lr: 1.000e-04, loss 0.244271
Stage: train, iter: 3060, lr: 1.000e-04, loss 0.280256
Stage: train, iter: 3080, lr: 1.000e-04, loss 0.199193
Stage: train, iter: 3100, lr: 1.000e-04, loss 0.267195
Stage: train, iter: 3120, lr: 1.000e-04, loss 0.247157
Stage: train, iter: 3140, lr: 1.000e-04, loss 0.326304
Stage: train, iter: 3160, lr: 1.000e-04, loss 0.217323
Stage: train, iter: 3180, lr: 1.000e-04, loss 0.315515
Stage: train, iter: 3200, lr: 1.000e-04, loss 0.198533
Stage: train, iter: 3220, lr: 1.000e-04, loss 0.249973
Stage: train, iter: 3240, lr: 1.000e-04, loss 0.246101
Stage: train, iter: 3260, lr: 1.000e-04, loss 0.280291
Stage: train, iter: 3280, lr: 1.000e-04, loss 0.303512
Stage: train, iter: 3300, lr: 1.000e-04, loss 0.224586
Stage: train, iter: 3320, lr: 1.000e-04, loss 0.254621
Stage: train, iter: 3340, lr: 1.000e-04, loss 0.258605
Stage: tra

Stage: train, iter: 5980, lr: 1.000e-04, loss 0.157280
Stage: train, iter: 6000, lr: 1.000e-04, loss 0.177862
Stage: train, iter: 6020, lr: 1.000e-04, loss 0.160968
Stage: train, iter: 6040, lr: 1.000e-04, loss 0.231953
Stage: train, iter: 6060, lr: 1.000e-04, loss 0.196589
Stage: train, iter: 6080, lr: 1.000e-04, loss 0.330248
Stage: train, iter: 6100, lr: 1.000e-04, loss 0.196944
Stage: train, iter: 6120, lr: 1.000e-04, loss 0.164692
Stage: train, iter: 6140, lr: 1.000e-04, loss 0.198198
Stage: train, iter: 6160, lr: 1.000e-04, loss 0.223802
Stage: train, iter: 6180, lr: 1.000e-04, loss 0.260657
Stage: train, iter: 6200, lr: 1.000e-04, loss 0.235499
Stage: train, iter: 6220, lr: 1.000e-04, loss 0.191748
Stage: train, iter: 6240, lr: 1.000e-04, loss 0.218095
Stage: train, iter: 6260, lr: 1.000e-04, loss 0.161784
Stage: train, iter: 6280, lr: 1.000e-04, loss 0.250515
Stage: train, iter: 6300, lr: 1.000e-04, loss 0.205006
Stage: train, iter: 6320, lr: 1.000e-04, loss 0.322128
Stage: tra

Stage: train, iter: 8960, lr: 1.000e-04, loss 0.259526
Stage: train, iter: 8980, lr: 1.000e-04, loss 0.177447
Stage: train, iter: 9000, lr: 1.000e-04, loss 0.189899
Stage: train, iter: 9020, lr: 1.000e-04, loss 0.175669
Stage: train, iter: 9040, lr: 1.000e-04, loss 0.199418
Stage: train, iter: 9060, lr: 1.000e-04, loss 0.198282
Stage: train, iter: 9080, lr: 1.000e-04, loss 0.348602
Stage: train, iter: 9100, lr: 1.000e-04, loss 0.145196
Stage: train, iter: 9120, lr: 1.000e-04, loss 0.233472
Stage: train, iter: 9140, lr: 1.000e-04, loss 0.208053
Stage: train, iter: 9160, lr: 1.000e-04, loss 0.214510
Stage: train, iter: 9180, lr: 1.000e-04, loss 0.177183
Stage: train, iter: 9200, lr: 1.000e-04, loss 0.401374
Stage: train, iter: 9220, lr: 1.000e-04, loss 0.153557
Stage: train, iter: 9240, lr: 1.000e-04, loss 0.202854
Stage: train, iter: 9260, lr: 1.000e-04, loss 0.160881
Stage: train, iter: 9280, lr: 1.000e-04, loss 0.262511
Stage: train, iter: 9300, lr: 1.000e-04, loss 0.178704
Stage: tra

In [17]:
experiment_name = 'res50_i512_ch128_dice0.5_400k'
experiment_dir = OUTPUT_DIR + experiment_name
#!rm -r {experiment_dir}
!mkdir -p {experiment_dir}

In [ ]:
logger = Logger(pjoin(experiment_dir, 'train_log.txt'))

model = get_model()
model.load_state_dict(torch.load('./output/res50_i512_ch128_dice0.5/model_180000.pt'))

start_lr = 1e-4
optimizer = Adam(model.parameters(), lr=start_lr)
# lr_policy = StepLR(1e-5, 20000, lr_decay=0.5)
lr_policy = CyclicLR(start_lr, 20000)
criterion = BCEDiceLoss(w_bce=1, w_dice=0.5)

train_dict = {
    'model': model,
    'output_dirpath': experiment_dir,
    'optimizer': optimizer,
    'lr_policy': lr_policy,
#     'start_iter': 200000,
    'num_iters': 200000,
    'criterion': criterion,
    'train_dataloader': dl_train,
    'val_set': val_set,
    'logger': logger,
}
train_hist = train(**train_dict)

model_dirpath = join(experiment_dir, f'model_last.pt')
torch.save(model.state_dict(), model_dirpath)

Stage: train, iter: 0, lr: 1.000e-04, loss 0.157827
Stage: train, iter: 20, lr: 1.000e-04, loss 0.135674
Stage: train, iter: 40, lr: 1.000e-04, loss 0.159892
Stage: train, iter: 60, lr: 1.000e-04, loss 0.164676
Stage: train, iter: 80, lr: 1.000e-04, loss 0.167141
Stage: train, iter: 100, lr: 9.999e-05, loss 0.133138
Stage: train, iter: 120, lr: 9.999e-05, loss 0.125994
Stage: train, iter: 140, lr: 9.999e-05, loss 0.125550
Stage: train, iter: 160, lr: 9.998e-05, loss 0.157842
Stage: train, iter: 180, lr: 9.998e-05, loss 0.114782
Stage: train, iter: 200, lr: 9.998e-05, loss 0.099650
Stage: train, iter: 220, lr: 9.997e-05, loss 0.137164
Stage: train, iter: 240, lr: 9.996e-05, loss 0.112647
Stage: train, iter: 260, lr: 9.996e-05, loss 0.125325
Stage: train, iter: 280, lr: 9.995e-05, loss 0.133991
Stage: train, iter: 300, lr: 9.994e-05, loss 0.075667
Stage: train, iter: 320, lr: 9.994e-05, loss 0.154214
Stage: train, iter: 340, lr: 9.993e-05, loss 0.089717
Stage: train, iter: 360, lr: 9.992

Stage: train, iter: 3000, lr: 9.455e-05, loss 0.116062
Stage: train, iter: 3020, lr: 9.448e-05, loss 0.115118
Stage: train, iter: 3040, lr: 9.441e-05, loss 0.132274
Stage: train, iter: 3060, lr: 9.433e-05, loss 0.102333
Stage: train, iter: 3080, lr: 9.426e-05, loss 0.119889
Stage: train, iter: 3100, lr: 9.419e-05, loss 0.160934
Stage: train, iter: 3120, lr: 9.411e-05, loss 0.070023
Stage: train, iter: 3140, lr: 9.404e-05, loss 0.107824
Stage: train, iter: 3160, lr: 9.397e-05, loss 0.153217
Stage: train, iter: 3180, lr: 9.389e-05, loss 0.105700
Stage: train, iter: 3200, lr: 9.381e-05, loss 0.113188
Stage: train, iter: 3220, lr: 9.374e-05, loss 0.146792
Stage: train, iter: 3240, lr: 9.366e-05, loss 0.111069
Stage: train, iter: 3260, lr: 9.359e-05, loss 0.156894
Stage: train, iter: 3280, lr: 9.351e-05, loss 0.147990
Stage: train, iter: 3300, lr: 9.343e-05, loss 0.144896
Stage: train, iter: 3320, lr: 9.335e-05, loss 0.192795
Stage: train, iter: 3340, lr: 9.327e-05, loss 0.119154
Stage: tra

Stage: train, iter: 5980, lr: 7.951e-05, loss 0.148939
Stage: train, iter: 6000, lr: 7.939e-05, loss 0.143242
Stage: train, iter: 6020, lr: 7.926e-05, loss 0.124544
Stage: train, iter: 6040, lr: 7.913e-05, loss 0.112902
Stage: train, iter: 6060, lr: 7.900e-05, loss 0.152296
Stage: train, iter: 6080, lr: 7.888e-05, loss 0.116851
Stage: train, iter: 6100, lr: 7.875e-05, loss 0.134065
Stage: train, iter: 6120, lr: 7.862e-05, loss 0.105512
Stage: train, iter: 6140, lr: 7.849e-05, loss 0.124802
Stage: train, iter: 6160, lr: 7.836e-05, loss 0.104386
Stage: train, iter: 6180, lr: 7.823e-05, loss 0.166698
Stage: train, iter: 6200, lr: 7.810e-05, loss 0.121399
Stage: train, iter: 6220, lr: 7.797e-05, loss 0.068773
Stage: train, iter: 6240, lr: 7.784e-05, loss 0.143039
Stage: train, iter: 6260, lr: 7.771e-05, loss 0.065096
Stage: train, iter: 6280, lr: 7.758e-05, loss 0.101115
Stage: train, iter: 6300, lr: 7.745e-05, loss 0.099639
Stage: train, iter: 6320, lr: 7.732e-05, loss 0.124931
Stage: tra

Stage: train, iter: 8960, lr: 5.813e-05, loss 0.090967
Stage: train, iter: 8980, lr: 5.797e-05, loss 0.086160
Stage: train, iter: 9000, lr: 5.782e-05, loss 0.164147
Stage: train, iter: 9020, lr: 5.766e-05, loss 0.133598
Stage: train, iter: 9040, lr: 5.751e-05, loss 0.108664
Stage: train, iter: 9060, lr: 5.735e-05, loss 0.118444
Stage: train, iter: 9080, lr: 5.720e-05, loss 0.077046
Stage: train, iter: 9100, lr: 5.704e-05, loss 0.155089
Stage: train, iter: 9120, lr: 5.689e-05, loss 0.088839
Stage: train, iter: 9140, lr: 5.673e-05, loss 0.119321
Stage: train, iter: 9160, lr: 5.657e-05, loss 0.127459
Stage: train, iter: 9180, lr: 5.642e-05, loss 0.109406
Stage: train, iter: 9200, lr: 5.626e-05, loss 0.175160
Stage: train, iter: 9220, lr: 5.611e-05, loss 0.173955
Stage: train, iter: 9240, lr: 5.595e-05, loss 0.116531
Stage: train, iter: 9260, lr: 5.580e-05, loss 0.093263
Stage: train, iter: 9280, lr: 5.564e-05, loss 0.126722
Stage: train, iter: 9300, lr: 5.548e-05, loss 0.104452
Stage: tra

In [23]:
experiment_name = 'res50_i512_ch128_dice0.5_train+val'
experiment_dir = OUTPUT_DIR + experiment_name
#!rm -r {experiment_dir}
!mkdir -p {experiment_dir}

In [26]:
ds_train = Dataset(IMAGE_SZ, train_set + val_set, epoch_mul=epoch_mul)
dl_train = DataLoader(ds_train, batch_size=batch_size, shuffle=True, num_workers=8)

In [27]:
logger = Logger(pjoin(experiment_dir, 'train_log.txt'))

model = get_model()
model.load_state_dict(torch.load('./output/res50_i512_ch128_dice0.5_400k/model_180000.pt'))

start_lr = 1e-5
optimizer = Adam(model.parameters(), lr=start_lr)
# lr_policy = StepLR(1e-5, 20000, lr_decay=0.5)
lr_policy = CyclicLR(start_lr, 20000)
criterion = BCEDiceLoss(w_bce=1, w_dice=0.5)

train_dict = {
    'model': model,
    'output_dirpath': experiment_dir,
    'optimizer': optimizer,
    'lr_policy': lr_policy,
#     'start_iter': 200000,
    'num_iters': 60000,
    'criterion': criterion,
    'train_dataloader': dl_train,
    'val_set': val_set,
    'logger': logger,
}
train_hist = train(**train_dict)

model_dirpath = join(experiment_dir, f'model_last.pt')
torch.save(model.state_dict(), model_dirpath)

Stage: train, iter: 0, lr: 1.000e-05, loss 0.063101
Stage: train, iter: 20, lr: 1.000e-05, loss 0.108421
Stage: train, iter: 40, lr: 1.000e-05, loss 0.091700
Stage: train, iter: 60, lr: 1.000e-05, loss 0.126670
Stage: train, iter: 80, lr: 1.000e-05, loss 0.098768
Stage: train, iter: 100, lr: 9.999e-06, loss 0.084430
Stage: train, iter: 120, lr: 9.999e-06, loss 0.054597
Stage: train, iter: 140, lr: 9.999e-06, loss 0.055674
Stage: train, iter: 160, lr: 9.998e-06, loss 0.139285
Stage: train, iter: 180, lr: 9.998e-06, loss 0.071402
Stage: train, iter: 200, lr: 9.998e-06, loss 0.109633
Stage: train, iter: 220, lr: 9.997e-06, loss 0.082866
Stage: train, iter: 240, lr: 9.996e-06, loss 0.103660
Stage: train, iter: 260, lr: 9.996e-06, loss 0.114150
Stage: train, iter: 280, lr: 9.995e-06, loss 0.099381
Stage: train, iter: 300, lr: 9.994e-06, loss 0.092966
Stage: train, iter: 320, lr: 9.994e-06, loss 0.083791
Stage: train, iter: 340, lr: 9.993e-06, loss 0.092606
Stage: train, iter: 360, lr: 9.992

Stage: train, iter: 3000, lr: 9.455e-06, loss 0.098736
Stage: train, iter: 3020, lr: 9.448e-06, loss 0.138734
Stage: train, iter: 3040, lr: 9.441e-06, loss 0.070496
Stage: train, iter: 3060, lr: 9.433e-06, loss 0.110726
Stage: train, iter: 3080, lr: 9.426e-06, loss 0.075645
Stage: train, iter: 3100, lr: 9.419e-06, loss 0.091790
Stage: train, iter: 3120, lr: 9.411e-06, loss 0.070406
Stage: train, iter: 3140, lr: 9.404e-06, loss 0.090732
Stage: train, iter: 3160, lr: 9.397e-06, loss 0.076973
Stage: train, iter: 3180, lr: 9.389e-06, loss 0.094418
Stage: train, iter: 3200, lr: 9.381e-06, loss 0.086077
Stage: train, iter: 3220, lr: 9.374e-06, loss 0.079238
Stage: train, iter: 3240, lr: 9.366e-06, loss 0.086994
Stage: train, iter: 3260, lr: 9.359e-06, loss 0.073030
Stage: train, iter: 3280, lr: 9.351e-06, loss 0.062822
Stage: train, iter: 3300, lr: 9.343e-06, loss 0.134609
Stage: train, iter: 3320, lr: 9.335e-06, loss 0.078562
Stage: train, iter: 3340, lr: 9.327e-06, loss 0.109214
Stage: tra

Stage: train, iter: 5980, lr: 7.951e-06, loss 0.078210
Stage: train, iter: 6000, lr: 7.939e-06, loss 0.077891
Stage: train, iter: 6020, lr: 7.926e-06, loss 0.123904
Stage: train, iter: 6040, lr: 7.913e-06, loss 0.070642
Stage: train, iter: 6060, lr: 7.900e-06, loss 0.033895
Stage: train, iter: 6080, lr: 7.888e-06, loss 0.065946
Stage: train, iter: 6100, lr: 7.875e-06, loss 0.067805
Stage: train, iter: 6120, lr: 7.862e-06, loss 0.074429
Stage: train, iter: 6140, lr: 7.849e-06, loss 0.080810
Stage: train, iter: 6160, lr: 7.836e-06, loss 0.058845
Stage: train, iter: 6180, lr: 7.823e-06, loss 0.085284
Stage: train, iter: 6200, lr: 7.810e-06, loss 0.063423
Stage: train, iter: 6220, lr: 7.797e-06, loss 0.066719
Stage: train, iter: 6240, lr: 7.784e-06, loss 0.112648
Stage: train, iter: 6260, lr: 7.771e-06, loss 0.083744
Stage: train, iter: 6280, lr: 7.758e-06, loss 0.103942
Stage: train, iter: 6300, lr: 7.745e-06, loss 0.074743
Stage: train, iter: 6320, lr: 7.732e-06, loss 0.111440
Stage: tra

Stage: train, iter: 8960, lr: 5.813e-06, loss 0.088594
Stage: train, iter: 8980, lr: 5.797e-06, loss 0.040001
Stage: train, iter: 9000, lr: 5.782e-06, loss 0.101004
Stage: train, iter: 9020, lr: 5.766e-06, loss 0.078998
Stage: train, iter: 9040, lr: 5.751e-06, loss 0.086736
Stage: train, iter: 9060, lr: 5.735e-06, loss 0.080655
Stage: train, iter: 9080, lr: 5.720e-06, loss 0.076050
Stage: train, iter: 9100, lr: 5.704e-06, loss 0.058134
Stage: train, iter: 9120, lr: 5.689e-06, loss 0.080617
Stage: train, iter: 9140, lr: 5.673e-06, loss 0.077389
Stage: train, iter: 9160, lr: 5.657e-06, loss 0.073424
Stage: train, iter: 9180, lr: 5.642e-06, loss 0.124537
Stage: train, iter: 9200, lr: 5.626e-06, loss 0.057966
Stage: train, iter: 9220, lr: 5.611e-06, loss 0.048729
Stage: train, iter: 9240, lr: 5.595e-06, loss 0.093172
Stage: train, iter: 9260, lr: 5.580e-06, loss 0.087205
Stage: train, iter: 9280, lr: 5.564e-06, loss 0.086633
Stage: train, iter: 9300, lr: 5.548e-06, loss 0.111675
Stage: tra

Stage: train, iter: 11920, lr: 3.514e-06, loss 0.078467
Stage: train, iter: 11940, lr: 3.499e-06, loss 0.075960
Stage: train, iter: 11960, lr: 3.484e-06, loss 0.085225
Stage: train, iter: 11980, lr: 3.469e-06, loss 0.115077
Stage: train, iter: 12000, lr: 3.454e-06, loss 0.094082
Stage: train, iter: 12020, lr: 3.440e-06, loss 0.133628
Stage: train, iter: 12040, lr: 3.425e-06, loss 0.073088
Stage: train, iter: 12060, lr: 3.410e-06, loss 0.099480
Stage: train, iter: 12080, lr: 3.395e-06, loss 0.097577
Stage: train, iter: 12100, lr: 3.380e-06, loss 0.068968
Stage: train, iter: 12120, lr: 3.365e-06, loss 0.113276
Stage: train, iter: 12140, lr: 3.350e-06, loss 0.117116
Stage: train, iter: 12160, lr: 3.335e-06, loss 0.108316
Stage: train, iter: 12180, lr: 3.321e-06, loss 0.050909
Stage: train, iter: 12200, lr: 3.306e-06, loss 0.071628
Stage: train, iter: 12220, lr: 3.291e-06, loss 0.130169
Stage: train, iter: 12240, lr: 3.276e-06, loss 0.067279
Stage: train, iter: 12260, lr: 3.262e-06, loss 0

...

Stage: train, iter: 47200, lr: 7.129e-06, loss 0.077370
Stage: train, iter: 47220, lr: 7.114e-06, loss 0.057413
Stage: train, iter: 47240, lr: 7.100e-06, loss 0.066232
Stage: train, iter: 47260, lr: 7.086e-06, loss 0.060371
Stage: train, iter: 47280, lr: 7.072e-06, loss 0.024611
Stage: train, iter: 47300, lr: 7.057e-06, loss 0.056347
Stage: train, iter: 47320, lr: 7.043e-06, loss 0.068898
Stage: train, iter: 47340, lr: 7.029e-06, loss 0.071884
Stage: train, iter: 47360, lr: 7.014e-06, loss 0.063999
Stage: train, iter: 47380, lr: 7.000e-06, loss 0.078558
Stage: train, iter: 47400, lr: 6.985e-06, loss 0.071884
Stage: train, iter: 47420, lr: 6.971e-06, loss 0.085310
Stage: train, iter: 47440, lr: 6.957e-06, loss 0.123534
Stage: train, iter: 47460, lr: 6.942e-06, loss 0.046830
Stage: train, iter: 47480, lr: 6.928e-06, loss 0.094018
Stage: train, iter: 47500, lr: 6.913e-06, loss 0.102429
Stage: train, iter: 47520, lr: 6.899e-06, loss 0.098802
Stage: train, iter: 47540, lr: 6.884e-06, loss 0

Stage: train, iter: 50140, lr: 4.890e-06, loss 0.091579
Stage: train, iter: 50160, lr: 4.874e-06, loss 0.070369
Stage: train, iter: 50180, lr: 4.858e-06, loss 0.061537
Stage: train, iter: 50200, lr: 4.843e-06, loss 0.081559
Stage: train, iter: 50220, lr: 4.827e-06, loss 0.076340
Stage: train, iter: 50240, lr: 4.811e-06, loss 0.078138
Stage: train, iter: 50260, lr: 4.795e-06, loss 0.096660
Stage: train, iter: 50280, lr: 4.780e-06, loss 0.072315
Stage: train, iter: 50300, lr: 4.764e-06, loss 0.077976
Stage: train, iter: 50320, lr: 4.748e-06, loss 0.055112
Stage: train, iter: 50340, lr: 4.733e-06, loss 0.095233
Stage: train, iter: 50360, lr: 4.717e-06, loss 0.084264
Stage: train, iter: 50380, lr: 4.701e-06, loss 0.070222
Stage: train, iter: 50400, lr: 4.686e-06, loss 0.064683
Stage: train, iter: 50420, lr: 4.670e-06, loss 0.091993
Stage: train, iter: 50440, lr: 4.654e-06, loss 0.079700
Stage: train, iter: 50460, lr: 4.639e-06, loss 0.067255
Stage: train, iter: 50480, lr: 4.623e-06, loss 0

Stage: train, iter: 53080, lr: 2.674e-06, loss 0.093483
Stage: train, iter: 53100, lr: 2.660e-06, loss 0.062688
Stage: train, iter: 53120, lr: 2.646e-06, loss 0.104165
Stage: train, iter: 53140, lr: 2.632e-06, loss 0.093298
Stage: train, iter: 53160, lr: 2.618e-06, loss 0.098458
Stage: train, iter: 53180, lr: 2.605e-06, loss 0.056547
Stage: train, iter: 53200, lr: 2.591e-06, loss 0.070906
Stage: train, iter: 53220, lr: 2.577e-06, loss 0.050430
Stage: train, iter: 53240, lr: 2.563e-06, loss 0.087760
Stage: train, iter: 53260, lr: 2.550e-06, loss 0.087235
Stage: train, iter: 53280, lr: 2.536e-06, loss 0.065433
Stage: train, iter: 53300, lr: 2.522e-06, loss 0.100367
Stage: train, iter: 53320, lr: 2.509e-06, loss 0.086758
Stage: train, iter: 53340, lr: 2.495e-06, loss 0.061662
Stage: train, iter: 53360, lr: 2.481e-06, loss 0.047210
Stage: train, iter: 53380, lr: 2.468e-06, loss 0.095966
Stage: train, iter: 53400, lr: 2.454e-06, loss 0.087672
Stage: train, iter: 53420, lr: 2.441e-06, loss 0

Stage: train, iter: 56020, lr: 9.453e-07, loss 0.078387
Stage: train, iter: 56040, lr: 9.362e-07, loss 0.118725
Stage: train, iter: 56060, lr: 9.270e-07, loss 0.083216
Stage: train, iter: 56080, lr: 9.179e-07, loss 0.080509
Stage: train, iter: 56100, lr: 9.089e-07, loss 0.075106
Stage: train, iter: 56120, lr: 8.999e-07, loss 0.086598
Stage: train, iter: 56140, lr: 8.909e-07, loss 0.092820
Stage: train, iter: 56160, lr: 8.820e-07, loss 0.074026
Stage: train, iter: 56180, lr: 8.731e-07, loss 0.081283
Stage: train, iter: 56200, lr: 8.642e-07, loss 0.065454
Stage: train, iter: 56220, lr: 8.554e-07, loss 0.107913
Stage: train, iter: 56240, lr: 8.467e-07, loss 0.095013
Stage: train, iter: 56260, lr: 8.379e-07, loss 0.074757
Stage: train, iter: 56280, lr: 8.293e-07, loss 0.083218
Stage: train, iter: 56300, lr: 8.206e-07, loss 0.066662
Stage: train, iter: 56320, lr: 8.120e-07, loss 0.074153
Stage: train, iter: 56340, lr: 8.034e-07, loss 0.053412
Stage: train, iter: 56360, lr: 7.949e-07, loss 0

Stage: train, iter: 58960, lr: 6.645e-08, loss 0.051945
Stage: train, iter: 58980, lr: 6.392e-08, loss 0.066936
Stage: train, iter: 59000, lr: 6.144e-08, loss 0.056959
Stage: train, iter: 59020, lr: 5.901e-08, loss 0.082397
Stage: train, iter: 59040, lr: 5.663e-08, loss 0.105391
Stage: train, iter: 59060, lr: 5.430e-08, loss 0.083896
Stage: train, iter: 59080, lr: 5.201e-08, loss 0.057992
Stage: train, iter: 59100, lr: 4.978e-08, loss 0.083953
Stage: train, iter: 59120, lr: 4.759e-08, loss 0.085364
Stage: train, iter: 59140, lr: 4.545e-08, loss 0.058849
Stage: train, iter: 59160, lr: 4.336e-08, loss 0.064490
Stage: train, iter: 59180, lr: 4.132e-08, loss 0.093415
Stage: train, iter: 59200, lr: 3.933e-08, loss 0.053591
Stage: train, iter: 59220, lr: 3.739e-08, loss 0.064750
Stage: train, iter: 59240, lr: 3.550e-08, loss 0.096881
Stage: train, iter: 59260, lr: 3.365e-08, loss 0.086174
Stage: train, iter: 59280, lr: 3.186e-08, loss 0.069991
Stage: train, iter: 59300, lr: 3.011e-08, loss 0

In [24]:
logger.log.flush()